In [1]:
from os import getenv
from dotenv import load_dotenv
from pyspark.sql import SparkSession
import pyspark.sql.functions as F

In [2]:
load_dotenv("../dags/.env")

True

In [3]:
AWS_PACKAGES = "org.apache.hadoop:hadoop-aws:3.3.1,com.amazonaws:aws-java-sdk-bundle:1.11.901"
DELTA_PACKAGES = "io.delta:delta-core_2.12:2.4.0,io.delta:delta-storage:2.4.0"

spark = (
    SparkSession
    .builder
    .config("spark.jars.packages", f"{AWS_PACKAGES},{DELTA_PACKAGES}")
    .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
    .config("spark.hadoop.fs.s3a.access.key", getenv("access_key"))
    .config("spark.hadoop.fs.s3a.secret.key", getenv("secret_access_key"))
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")
    .config("spark.hadoop.fs.s3a.aws.credentials.provider", "org.apache.hadoop.fs.s3a.SimpleAWSCredentialsProvider")
    .getOrCreate()
)

23/06/15 21:48:01 WARN Utils: Your hostname, nobara-pc resolves to a loopback address: 127.0.1.1; using 192.168.15.132 instead (on interface wlp4s0)
23/06/15 21:48:01 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/home/lucasm/Github/projects/govbr-scrapy-data/.venv/lib/python3.10/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/lucasm/.ivy2/cache
The jars for the packages stored in: /home/lucasm/.ivy2/jars
org.apache.hadoop#hadoop-aws added as a dependency
com.amazonaws#aws-java-sdk-bundle added as a dependency
io.delta#delta-core_2.12 added as a dependency
io.delta#delta-storage added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-31a09cdb-b42c-4bcf-a93b-cb58efdfad6e;1.0
	confs: [default]
	found org.apache.hadoop#hadoop-aws;3.3.1 in central
	found com.amazonaws#aws-java-sdk-bundle;1.11.901 in central
	found org.wildfly.openssl#wildfly-openssl;1.0.7.Final in central
	found io.delta#delta-core_2.12;2.4.0 in central
	found io.delta#delta-storage;2.4.0 in central
	found org.antlr#antlr4-runtime;4.9.3 in central
downloading https://repo1.maven.org/maven2/org/apache/hadoop/hadoop-aws/3.3.1/hadoop-aws-3.3.1.jar ...
	[SUCCESSFUL ] org.apache.hadoop#hadoop-aws;3.3.1!hadoop-aws.jar (692ms)
downloading https://repo1.maven.org/maven2/com/amazonaws/aws-ja

In [10]:
metadata_df = spark.read.format("delta").load("s3a://govbr-data/bronze/metadata/")
metadata_df.show(10, truncate=False)

+----------------------------------------------------------------------------------------------------+-------------------+------+--------------------------+--------------+----------------------+
|url                                                                                                 |display_name       |source|downloaded_at             |filename      |datasource            |
+----------------------------------------------------------------------------------------------------+-------------------+------+--------------------------+--------------+----------------------+
|https://www.gov.br/anp/pt-br/centrais-de-conteudo/dados-abertos/arquivos/shpc/dsas/ca/ca-2015-01.csv|1º semestre de 2015|ANP   |2023-06-15 22:18:52.189621|ca-2015-01.csv|precos-de-combustiveis|
|https://www.gov.br/anp/pt-br/centrais-de-conteudo/dados-abertos/arquivos/shpc/dsas/ca/ca-2020-01.csv|1º semestre de 2020|ANP   |2023-06-15 22:12:34.396317|ca-2020-01.csv|precos-de-combustiveis|
+------------------------

In [5]:

already_downloaded_files = (
    # metadata_df.filter(F.col("url") == "https://www.gov.br/anp/pt-br/centrais-de-conteudo/dados-abertos/arquivos/shpc/dsas/ca/ca-2019-01.csv")
    metadata_df.select("display_name")
    .rdd.flatMap(lambda x: x).collect()
)
print(f"Already downloaded files: {already_downloaded_files}")

+----------------------------------------------------------------------------------------------------+-------------------+------+--------------------------+--------------+----------------------+
|url                                                                                                 |display_name       |source|downloaded_at             |filename      |datasource            |
+----------------------------------------------------------------------------------------------------+-------------------+------+--------------------------+--------------+----------------------+
|https://www.gov.br/anp/pt-br/centrais-de-conteudo/dados-abertos/arquivos/shpc/dsas/ca/ca-2022-02.zip|2º semestre de 2022|ANP   |2023-06-14 22:10:36.714282|ca-2022-02.csv|precos-de-combustiveis|
+----------------------------------------------------------------------------------------------------+-------------------+------+--------------------------+--------------+----------------------+

Already downloaded files